### Graph transformation history

In [ ]:
import nographs as nog
from graphviz import Digraph
import functools
import ast

In [ ]:
dict_2= {0:{1,}, 1:{2,3}, 2:{4,}, 3:{}, 4:{} }

In [ ]:
def to_id(x):
    return f"{(str( x ))}"

node_style =dict( style='filled', shape='box', 
                                align='left',
                                fontsize='12',
                                ranksep='0.01',
                                height='0.02',width='0.04',)
gr_style = dict( linelength="16",rankdir='LR',size='16,8')#landscape="True",splines='curved' ,layout="neato"

def adapter_graphviz( wrapped ):
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @functools.wraps( wrapped )
    def wrapper_bg( x, t ):
        node = dot.node( to_id(x), str(x) )
        for result in wrapped(x, t):
            dot.edge( to_id(x), to_id( result ), )# label=f"{str(x)}-{str(result)}",
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg
    

In [ ]:
class GraphMorph:
    def __init__(self, wrapped):
        functools.update_wrapper(self, wrapped )
    def __call__(self, x, t):
        yield from self.__wrapped__( x, t )

class GraphMorphDecorateNodes(GraphMorph):
    def __init__(self,*argc,**argv):
        super().__init__(*argc,**argv)
    @staticmethod
    def dress( x,):
        return x
    @staticmethod
    def undress( x,):
        return x
    def __call__(self, x, t):
        yield from map( self.dress, self.__wrapped__( self.undress(x), t ) )

class DecoName(GraphMorphDecorateNodes):
    def __init__(self,*argc,**argv):
        super().__init__(*argc,**argv)
    @staticmethod
    def dress( x,):
        return f"-{x}-"
    @staticmethod
    def undress( x,):
        return ast.literal_eval(x[1:-1])

In [ ]:
class FreeForget(GraphMorph):  # All possible paths
    def __init__(self,*argc,**argv):        super().__init__(*argc,**argv)
        
    def __call__(self, x, t):
        yield x # identity morphism
        for ch1 in self.__wrapped__( x, t ):
            yield from self.__call__(ch1, t) # recursion

In [ ]:
class IDFunctor(GraphMorphDecorateNodes):
    def __init__(self,*argc,**argv):
        super().__init__(*argc,**argv)
    @staticmethod
    def dress( x,):
        return f"-{x}-"
    @staticmethod
    def undress( x,):
        return ast.literal_eval(x[1:-1])
    def __call__(self, x, t):
        if type(x) is int:
            yield from                  self.__wrapped__(              x,  t )
            yield self.dress(x)
        else:
            yield from map( self.dress, self.__wrapped__( self.undress(x), t ) )
            #yield from                  self.__wrapped__( self.undress(x), t )

In [ ]:
class FreeForgetFunctor(GraphMorphDecorateNodes):
    def __init__(self, wrapped ):
        super().__init__(wrapped)
    @staticmethod
    def dress( x,):
        return f"-{x}-"
    @staticmethod
    def undress( x,):
        return ast.literal_eval(x[1:-1])
    def ff(self, x, t):
        yield x # identity morphism
        for ch1 in self.__wrapped__( x, t ):
            yield from self.ff(ch1, t) # recursion        
    def __call__(self, x, t):
        if type(x) is int:
            yield from                  self.__wrapped__(              x,  t )
            yield self.dress(x)
        else:
            yield from map( self.dress, self.ff( self.undress(x), t ) )

In [ ]:
class graphviz_bipart(  ):
    def __init__(self, predicate ):
        self.dot = {}
        self.dot[(False,False)] = Digraph( name="cluster_0" )
        self.dot[(True, True) ] = Digraph( name="cluster_1" )
        self.dot[(False,True) ] = Digraph( name='master')
        self.p = predicate
    def finalize(self):
        self.dot[(False,True) ].subgraph(self.dot[(False,False)])
        self.dot[(False,True) ].subgraph(self.dot[(True, True) ])
        return  self
    def get_dot(self):
        return  self.finalize().dot[(False,True) ]    
    def __call__( self, to_wrap ):
        @functools.wraps(to_wrap)
        def wrapper( x,t ):
            node = self.dot[self.p(x),self.p(x)].node( to_id(x), str(x) )
            for result in to_wrap(x, t):
                colour = 'black' if self.p(x) == self.p(result) else 'red'
                self.dot[(self.p(result),self.p(x))].edge( to_id(x), to_id(result), color=colour)
                yield result
        wrapper.get_dot = self.get_dot
        return wrapper

In [ ]:
def cat_dog():
    @graphviz_bipart( lambda x: type(x) is int )
    @FreeForgetFunctor
    def cat(x, _):
        return dict_2[x]
    return cat

cat_3 = cat_dog()
trav_forward = nog.TraversalBreadthFirst( cat_3 )
trav_forward.start_from( 0, )
print(str( list(trav_forward) ))
the_dot = cat_3.get_dot()
the_dot

In [ ]:
def bipart_the_dot( dot ):
    pass

In [ ]:
(the_dot.source)

In [ ]:
cat_3

In [ ]:
cat_3.__dict__#__closure__[0].cell_contents.get_dot()

In [ ]:
cat_3.get_dot()

In [ ]:
def join( f, seq ):
    for x in seq:
        yield from f(x)

sum( join( range, (5,2,7)))